In [ ]:
import ollama
# from langchain_community.llms import Ollama
# from langchain.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_ollama import OllamaEmbeddings
 
# from langchain_core.prompts import PromptTemplate
# from langchain_core.output_parsers import StrOutputParser
 
 
model = OllamaLLM(model="llama3.3", base_url="https://aicenter.mahidol.ac.th/ml")
 
# respons = model.invoke("Tell me a joke")
from langchain_core.vectorstores import InMemoryVectorStore
 
embeddings = OllamaEmbeddings(model="mxbai-embed-large", base_url="https://aicenter.mahidol.ac.th/ml")
text = "LangChain is the framework for building context-aware reasoning applications"
 


In [4]:
vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)
 
# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()
 
# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")
 
# show the retrieved document's content
retrieved_documents[0].page_content

ResponseError: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx</center>
</body>
</html>
 (status code: 504)

In [24]:
import ollama
# from langchain_community.llms import Ollama
# from langchain.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain_community.vectorstores import PGVector 
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document 
 
# from langchain_core.prompts import PromptTemplate
# from langchain_core.output_parsers import StrOutputParser
 
 
model = OllamaLLM(model="llama3.2", base_url="http://localhost:11434")
 
# respons = model.invoke("Tell me a joke")
from langchain_core.vectorstores import InMemoryVectorStore
 
embeddings = OllamaEmbeddings(model="mxbai-embed-large", base_url="http://localhost:11434")
# --- 2. กำหนดค่าการเชื่อมต่อ PostgreSQL ---
# ค่าเหล่านี้ต้องตรงกับที่กำหนดใน docker-compose.yml
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost" # Docker Container แมปพอร์ตมาที่ localhost ของเครื่องคุณ
DB_PORT = "5432"
DB_NAME = "mydatabase"

# สร้าง connection string สำหรับ PGVector
CONNECTION_STRING = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# กำหนดชื่อคอลเลกชัน (Collection name) ใน PGVector
# PGVector จะใช้ตาราง 'documents' ที่เราสร้างขึ้น และ 'collection_name' จะเป็นตัวแบ่งกลุ่มข้อมูล
COLLECTION_NAME = "my_ollama_documents_collection"

# --- 3. เตรียมข้อมูลที่จะทำ Embedding และเก็บ ---
documents_to_embed = [
    "LangChain is the framework for building context-aware reasoning applications",
    "A cat is a domestic species of small carnivorous mammal.",
    "A dog is a domesticated carnivorous mammal. It is the descendant of the wolf.",
    "Teachers can edit their account information by following these steps: 1. Log in to the system and select the Account menu (as shown in the example image). The system will display your personal information page. 2. Edit the desired information, then click the Save button at the bottom of the page. 3. Log out and log back in to the system. After that, you can proceed to print the certificate as usual.",
    "คุณครูสามารถแก้ไขได้ด้วยตนเองค่ะ ทำตามขั้นตอนดังนี้ n/1 ล็อกอินเข้าระบบ แล้วมาที่เมนู บัญชี ดังรูป จากนั้นระบบจะแสดงหน้าข้อมูลส่วนตัวค่ะ /n2 ทำการแก้ไขข้อมูลอีเมลให้ตรงกับระบบ ENDB จากนั้นกดบันทึก ปุ่มด้านล่างค่ะ และกดปิดหน้าข้อมูลส่วนตัว /n3 เมื่อแก้ไขอีเมลเรียบร้อยแล้ว คุณครูสามารถเข้าระบบ ENDB ด้วยอีเมลเดิมได้เลยค่ะ"
]

# แปลงข้อความเป็น LangChain Document object
docs = [Document(page_content=text) for text in documents_to_embed]

# --- 4. สร้าง Vector Store และบันทึกข้อมูลลง PostgreSQL ---
print("Connecting to PGVector store and adding documents...")
# 'from_documents' จะทำการ Embedding ข้อความและบันทึกข้อมูลลง DB ให้โดยอัตโนมัติ
# 'pre_delete_collection=False' เพื่อไม่ให้ลบข้อมูลเก่าทุกครั้งที่รัน
vectorstore = PGVector.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True # ตั้งค่าเป็น True ถ้าต้องการลบข้อมูลเก่าใน collection ก่อนเพิ่มใหม่
)
print(f"Documents added to PostgreSQL in collection '{COLLECTION_NAME}'.")

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()
 
# Retrieve the most similar text
retrieved_documents = retriever.invoke("i want to edit account")
 
# show the retrieved document's content
retrieved_documents[0].page_content

Connecting to PGVector store and adding documents...


c:\Users\stamp7ven\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\vectorstores\pgvector.py:490: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update your queries to use the new operators. 
  store = cls(


Documents added to PostgreSQL in collection 'my_ollama_documents_collection'.


'Teachers can edit their account information by following these steps: 1. Log in to the system and select the Account menu (as shown in the example image). The system will display your personal information page. 2. Edit the desired information, then click the Save button at the bottom of the page. 3. Log out and log back in to the system. After that, you can proceed to print the certificate as usual.'